In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [4]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/factRuEval-2016/dev.csv",
    valid_df_path="/home/eartemov/ae/work/factRuEval-2016/test.csv",
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMNCRF

In [6]:
model = BERTBiLSTMNCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, nbest=7)

build CRF...


In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMNCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2235259

In [12]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 50.


INFO:root:
epoch 51, average train epoch loss=0.0015935



INFO:root:on epoch 38 by max_f1: 0.89


              precision    recall  f1-score   support

         I_O      0.973     0.989     0.981     48875
       I_LOC      0.884     0.788     0.833      1557
       I_PER      0.955     0.925     0.939      2112
       I_ORG      0.874     0.695     0.775      3865

   micro avg      0.964     0.961     0.963     56409
   macro avg      0.921     0.849     0.882     56409
weighted avg      0.963     0.961     0.961     56409



### Eval

#### Load model

In [13]:
from modules.data import bert_data

In [15]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/factRuEval-2016/dev.csv",
    valid_df_path="/home/eartemov/ae/work/factRuEval-2016/test.csv",
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [16]:
from modules.models.bert_models import BERTBiLSTMNCRF

In [17]:
model = BERTBiLSTMNCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, nbest=7)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpxxd4ym1l
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

build CRF...


In [22]:
from modules.train.train import NerLearner

In [25]:
learner = NerLearner(model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMNCRF-IO.cpt",
                     t_total=num_epochs * len(data.train_dl))

In [26]:
model.get_n_trainable_params()

2235259

In [27]:
learner.load_model()

### Predict

In [28]:
from modules.data.bert_data import get_data_loader_for_predict

In [29]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [30]:
preds = learner.predict(dl)

In [31]:
from sklearn_crfsuite.metrics import flat_classification_report

In [32]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [33]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [34]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [35]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8358    0.7545    0.7930      3865
       I_PER     0.9432    0.9508    0.9469      2112
       I_LOC     0.8908    0.7913    0.8381      1557

   micro avg     0.8793    0.8171    0.8471      7534
   macro avg     0.8899    0.8322    0.8594      7534
weighted avg     0.8773    0.8171    0.8455      7534



In [36]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [38]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7730    0.6955    0.7322      1307
           O     0.9727    0.9836    0.9782     48682
         ORG     0.6786    0.5466    0.6055      1877
         PER     0.8376    0.8048    0.8209      1301

    accuracy                         0.9568     53167
   macro avg     0.8155    0.7576    0.7842     53167
weighted avg     0.9541    0.9568    0.9551     53167

